<a href="https://colab.research.google.com/github/lorida0777/Ames_Housing_Data/blob/master/Recommendation_services_hospitaliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn pandas numpy matplotlib seaborn nltk spacy tqdm
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 85.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import os, re, time, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, classification_report, confusion_matrix,
                             ConfusionMatrixDisplay)
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments, EarlyStoppingCallback)
from datasets import Dataset as HFDataset

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Device : {DEVICE}")

✅ Device : cpu


In [4]:
SERVICES = ["Cardiologie","Urgences","Pédiatrie","Oncologie","Orthopédie","Neurologie"]

templates = {
    "Cardiologie": [
        "Hospitalisé pour douleurs thoraciques et arythmie cardiaque. ECG et bilan sanguin rapides. Le cardiologue a expliqué clairement mon traitement.",
        "Prise en charge excellente suite à mon infarctus. Pose d'un stent en urgence. Personnel compétent et rassurant.",
        "Suivi pour insuffisance cardiaque chronique. Échographies et holters bien organisés. Cardiologue à l'écoute.",
        "Consultation pour hypertension artérielle sévère. Ajustement du traitement antihypertenseur fait avec soin.",
        "Hospitalisation pour fibrillation auriculaire. Cardioversion réalisée dans de bonnes conditions.",
        "Mon père opéré du cœur ici. Équipe chirurgicale et cardiologues remarquables tout au long de la procédure.",
        "Douleurs poitrine et essoufflement : angine de poitrine diagnostiquée. Traitement bien expliqué.",
        "Pose d'un pacemaker. Tout s'est très bien déroulé, infirmières attentionnées, médecin professionnel.",
        "Suivi post chirurgie valvulaire. Rendez-vous de contrôle bien espacés, examens cardiaques coordonnés.",
        "Épreuve d'effort et coronarographie pour bilan de douleurs thoraciques. Résultats expliqués avec pédagogie.",
    ],
    "Urgences": [
        "Accueil aux urgences pour coupure profonde à la main. Attente 2h. Plaie suturée correctement.",
        "Arrivé en ambulance suite à une chute. Urgentistes réactifs pour écarter toute fracture. Scanner et radio bien réalisés.",
        "Nuit aux urgences pour forte fièvre et frissons. Soignants débordés mais prise en charge correcte.",
        "Mon enfant a avalé un corps étranger. Prise en charge immédiate. Personnel très professionnel.",
        "Intoxication alimentaire avec déshydratation. Perfusion posée rapidement. Hospitalisation courte.",
        "Accident de voiture, fracture suspectée. Immobilisation et douleur bien gérées en attendant l'orthopédiste.",
        "Crise d'asthme sévère aux urgences. Nébulisation et corticoïdes administrés rapidement.",
        "Douleur abdominale intense. Bilan et échographie en urgence. Appendicite diagnostiquée et opérée.",
        "Service très chargé mais triage efficace. Mon cas prioritaire traité sans délai excessif.",
        "Malaise avec perte de connaissance. Bilan complet réalisé avant orientation vers le bon service.",
    ],
    "Pédiatrie": [
        "Mon fils de 4 ans hospitalisé pour bronchiolite sévère. Équipe pédiatrique formidable, douce et rassurante.",
        "Ma fille traitée pour otite à répétition. Pose de drains sous anesthésie légère. Tout s'est bien passé.",
        "Hospitalisation pour déshydratation d'un nourrisson de 6 mois. Équipe rapide, parents au chevet.",
        "Suivi de croissance pour enfant avec retard psychomoteur. Service équipé, médecins à l'écoute.",
        "Convulsion fébrile de mon enfant. Prise en charge rapide et explications très claires.",
        "Service de néonatologie exceptionnel pour notre prématuré de 32 semaines. Équipe soignante remarquable.",
        "Gastroentérite avec vomissements chez mon bébé. Réhydratation orale bien supervisée.",
        "Allergie alimentaire sévère chez mon fils. Tests allergologiques réalisés en pédiatrie.",
        "Suivi oncologique pédiatrique pour leucémie. Équipe remarquable humainement.",
        "Pneumonie bilatérale chez une enfant de 7 ans. Antibiothérapie efficace, sortie après 5 jours.",
    ],
    "Oncologie": [
        "Suivi en oncologie pour cancer du sein. Chimiothérapie bien tolérée grâce à l'accompagnement.",
        "Immunothérapie pour mélanome métastatique. Suivi biologique et clinique rigoureux.",
        "Radiothérapie conformationnelle pour cancer de la prostate. Service bien organisé.",
        "Cancer colorectal avec métastases hépatiques. Protocole FOLFOX proposé par le staff pluridisciplinaire.",
        "Cure de chimiothérapie. Équipe infirmière professionnelle et bienveillante.",
        "Suivi post-chirurgical cancer pulmonaire. Scanners bien planifiés, résultats expliqués avec clarté.",
        "Consultation d'oncogénétique. Test BRCA réalisé. Suivi adapté à mon profil de risque.",
        "Lymphome hodgkinien traité par ABVD. Rémission obtenue après 6 cures.",
        "Greffe de moelle osseuse réalisée ici. Équipe de soins intensifs hématologiques au top.",
        "Essai clinique pour cancer du rein réfractaire. Reconnaissant envers l'équipe oncologique.",
    ],
    "Orthopédie": [
        "Opération du ménisque suite à blessure sportive. Chirurgie arthroscopique réussie. Rééducation en cours.",
        "Pose d'une prothèse totale de hanche. Récupération fonctionnelle rapide grâce à la kinésithérapie.",
        "Fracture du poignet traitée chirurgicalement avec pose d'une plaque. Excellent travail.",
        "Lombalgie chronique. IRM réalisée, hernie discale diagnostiquée, infiltration proposée.",
        "Chirurgie du canal carpien sous anesthésie locale. Fourmillements disparus dès le lendemain.",
        "Remplacement de prothèse de genou après descellement. Kinésithérapie débutée rapidement.",
        "Fracture complexe de la cheville avec ligaments. Ostéosynthèse réalisée. Suivi radiologique rigoureux.",
        "Scoliose sévère opérée par arthrodèse vertébrale. Résultat remarquable sur douleur et posture.",
        "Rupture du tendon d'Achille. Chirurgie réparatrice suivie d'une rééducation intensive.",
        "Douleurs d'épaule chroniques. Échographie et bilan articulaire complet. Infiltration proposée.",
    ],
    "Neurologie": [
        "Hospitalisation pour AVC ischémique. Thrombolyse réalisée dans les délais. Récupération en bonne voie.",
        "Suivi pour épilepsie pharmaco-résistante. EEG vidéo réalisé. Traitement antiépileptique ajusté.",
        "Consultation pour migraines chroniques invalidantes. Traitement de fond efficace trouvé.",
        "Diagnostic de sclérose en plaques. Équipe neurologique exemplaire dans l'annonce et le suivi.",
        "Bilan de démence débutante. Neuropsychologue, IRM, PET scan : bilan exhaustif bien expliqué.",
        "Maladie de Parkinson suivie ici. Neurologue ajuste le traitement régulièrement.",
        "Névralgie du trijumeau. Radiochirurgie stéréotaxique efficace. Douleurs réduites de 80%.",
        "EMG et étude de conduction nerveuse pour polyneuropathie diabétique. Résultats bien expliqués.",
        "Méningite bactérienne prise en charge en urgence. Antibiothérapie immédiate. Séquelles limitées.",
        "Suivi post-AVC avec rééducation. Équipe pluridisciplinaire kiné, ortho, orthophoniste coordonnée.",
    ]
}

prefixes = ["","Très satisfait : ","Dans l'ensemble, ","Suite à mon hospitalisation, ","Je témoigne : ",""]
suffixes = ["","Je recommande vivement."," Service de qualité."," Personnel dévoué.",""]

def generate_dataset(n_per_class=300):
    data = []
    for service, texts in templates.items():
        count = 0
        while count < n_per_class:
            avis = random.choice(prefixes) + random.choice(texts) + random.choice(suffixes)
            data.append({"texte": avis.strip(), "service": service})
            count += 1
    return pd.DataFrame(data).sample(frac=1, random_state=SEED).reset_index(drop=True)

df = generate_dataset(n_per_class=300)
print(f"✅ Dataset : {len(df)} avis")
print(df['service'].value_counts())
df.head()

✅ Dataset : 1800 avis
service
Neurologie     300
Oncologie      300
Pédiatrie      300
Cardiologie    300
Orthopédie     300
Urgences       300
Name: count, dtype: int64


,texte,service
0,Je témoigne : Hospitalisation pour AVC ischémi...,Neurologie
1,Suivi post-chirurgical cancer pulmonaire. Scan...,Oncologie
2,Gastroentérite avec vomissements chez mon bébé...,Pédiatrie
3,Très satisfait : Hospitalisation pour fibrilla...,Cardiologie
4,"Dans l'ensemble, Douleurs d'épaule chroniques....",Orthopédie
